## Bayesian updates using the table method

This notebook demonstrates a way of doing simple Bayesian updates using the table method, with a Pandas DataFrame as the table.

Copyright 2018 Allen Downey

MIT License: https://opensource.org/licenses/MIT


In [2]:
# Configure Jupyter so figures appear in the notebook
%matplotlib inline

# Configure Jupyter to display the assigned value after an assignment
%config InteractiveShell.ast_node_interactivity='last_expr_or_assign'

import numpy as np
import pandas as pd

As an example, I'll use the "cookie problem", which is a version of a classic probability "urn problem".

Suppose there are two bowls of cookies.

* Bowl #1 has 10 chocolate and 30 vanilla.

* Bowl #2 has 20 of each.

You choose a bowl at random, and then pick a cookie at random.  The cookie turns out to be vanilla.  What is the probability that the bowl you picked from is Bowl #1?

### The BayesTable class

Here's the class that represents a Bayesian table.

In [3]:
class BayesTable(pd.DataFrame):
    def __init__(self, hypo, prior=1):
        columns = ['hypo', 'prior', 'likelihood', 'unnorm', 'posterior']
        super().__init__(columns=columns)
        self.hypo = hypo
        self.prior = prior
    
    def mult(self):
        self.unnorm = self.prior * self.likelihood
        
    def norm(self):
        nc = np.sum(self.unnorm)
        self.posterior = self.unnorm / nc
        return nc
    
    def update(self):
        self.mult()
        return self.norm()
    
    def reset(self):
        return BayesTable(self.hypo, self.posterior)

Here's an instance that represents the two hypotheses: you either chose from Bowl 1 or Bowl 2:

In [19]:
chess = BayesTable(['AB','BA'])
chess.likelihood=[.4*.3,.7*.6]
chess.update()
chess

,hypo,prior,likelihood,unnorm,posterior
0,AB,1,0.12,0.12,0.222222
1,BA,1,0.42,0.42,0.777778


In [4]:
table = BayesTable([4,6,8,12])

,hypo,prior,likelihood,unnorm,posterior
0,4,1,NaN,NaN,NaN
1,6,1,NaN,NaN,NaN
2,8,1,NaN,NaN,NaN
3,12,1,NaN,NaN,NaN


Since we didn't specify prior probabilities, the default value is equal priors for all hypotheses.

Now we can specify the likelihoods:

* The likelihood of getting a vanilla cookie from Bowl 1 is 3/4.

* The likelihood of getting a vanilla cookie from Bowl 2 is 1/2.

Here's how we plug the likelihoods in:

In [5]:
table.likelihood = [1/4,1/6,1/8,1/12]
table

,hypo,prior,likelihood,unnorm,posterior
0,4,1,0.250000,NaN,NaN
1,6,1,0.166667,NaN,NaN
2,8,1,0.125000,NaN,NaN
3,12,1,0.083333,NaN,NaN


The next step is to multiply the priors by the likelihoods, which yields the unnormalized posteriors.

In [6]:
table.mult()
table

,hypo,prior,likelihood,unnorm,posterior
0,4,1,0.250000,0.250000,NaN
1,6,1,0.166667,0.166667,NaN
2,8,1,0.125000,0.125000,NaN
3,12,1,0.083333,0.083333,NaN


Now we can compute the normalized posteriors; `norm` returns the normalization constant.

In [7]:
table.norm()

0.625

In [8]:
table

,hypo,prior,likelihood,unnorm,posterior
0,4,1,0.250000,0.250000,0.400000
1,6,1,0.166667,0.166667,0.266667
2,8,1,0.125000,0.125000,0.200000
3,12,1,0.083333,0.083333,0.133333


We can read the posterior probabilities from the last column: the probability that you chose from Bowl 1 is 60%.

### Resetting

Suppose you put the first cookie back, stir the bowl, and select another cookie from the same bowl.

If this second cookie is chocolate, what is the probability, now, that you are drawing from Bowl 1?

To solve this problem, we want a new table where the priors in the new table are the posteriors from the old table.  That's what the `reset` method computes:

In [9]:
table2 = table.reset()

,hypo,prior,likelihood,unnorm,posterior
0,4,0.400000,NaN,NaN,NaN
1,6,0.266667,NaN,NaN,NaN
2,8,0.200000,NaN,NaN,NaN
3,12,0.133333,NaN,NaN,NaN


Here are the likelihoods for the second update.

In [10]:
table2.likelihood = [1/4,1/6,1/8,1/12]

We could run `mult` and `norm` again, or run `update`, which does both steps.

In [12]:
table2.mult()
table2.norm()

0.18055555555555552

Here are the results.

In [13]:
table2

,hypo,prior,likelihood,unnorm,posterior
0,4,0.400000,0.250000,0.100000,0.553846
1,6,0.266667,0.166667,0.044444,0.246154
2,8,0.200000,0.125000,0.025000,0.138462
3,12,0.133333,0.083333,0.011111,0.061538


But the result is the same.